# Klasyfikacja obrazu za pomocą konwolucyjnych sieci neuronowych 

Wykorzystamy konwolucyjne sieci neuronowe (ang. convolutional neural networks - CNNs), by nauczyć nasz komputer jak widzieć - a jest to coś, co jest możliwe jedynie dzięki głębokiemu uczeniu (ang. deep learning).

## Wprowadzenie do zadania "Old Polish Cars"

Spróbujemy stworzyć model, który pozwoli rozróżnić dwie marki starych polskich samochodów. Dostępne jest 2400 oznakowanych zdjęć samochodów do trenowania i 270 w zestawie testowym, które musimy spróbować oznakować.

In [0]:
# Umieść je na górze każdego notatnika, aby uzyskać automatyczne ponowne ładowanie
%reload_ext autoreload
%autoreload 2

#matplotlib setup
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import matplotlib.style
import matplotlib as mpl
mpl.style.use('default')
mpl.style.use('seaborn-ticks')
# %config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
import shutil as sh

Tutaj importujemy niezbędne biblioteki, do czego konkretnie które służą dowiemy się podczas kursu (http://course.fast.ai).

In [0]:
# cd fastai/

In [0]:
# Ten plik zawiera wszystkie główne biblioteki zewnętrzne, których użyjemy.
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

`PATH` jest ścieżką do twoich danych - jeśli wykorzystasz zalecaną metodę konfiguracji tej lekcji, nie będziesz musiał tego zmieniać. `sz` to rozmiar, do którego obrazy zostaną przeskalowane w celu uzyskania szybkich przebiegów treningowych. Podczas kursu będziemy sporo rozmawiać o tym parametrze. Zostaw na razie `224`.

In [0]:
PATH = "/content/data/old_polish_cars/old_polish_cars_two_classes_v2a-512-split/"
sz=224

Ważne jest, aby mieć działający GPU NVidii. Zestaw bibliotek stosowany za kulisami do pracy z procesorami graficznymi NVidii nosi nazwę CUDA. Dlatego zanim przejdzisz dalej, upewnij się, że następująca linia zwraca wartość `True`. Jeśli masz z tym problemy, sprawdź FAQ i poproś o pomoc [na forum](http://forums.fast.ai).

In [0]:
torch.cuda.is_available()

Ponadto NVidia zapewnia specjalne przyspieszone funkcje do głębokiego uczenia w pakiecie o nazwie CuDNN. Chociaż nie jest to bezwzględnie konieczne, znacznie poprawi wydajność treningów i jest domyślnie uwzględnione we wszystkich obsługiwanych konfiguracjach fastai. Dlatego jeśli poniższe nie zwróci wartości `True`, warto sprawdzić, dlaczego.

In [0]:
torch.backends.cudnn.enabled

## Najpierw obejrzyj zdjęcia samochodów

Biblioteka `fastai` zakłada, że posiadasz katalogi *train* i *valid*. Zakłada się także, że każdy katalog będzie zawierał podkatalogi dla każdej klasy, która ma zostać rozpoznana (w tym przypadku "maluch", "polonez" itp.).

In [0]:
os.listdir(PATH)

In [0]:
os.listdir(f'{PATH}valid')

In [0]:
files = os.listdir(f'{PATH}valid/maluch')[:5]
files

In [0]:
img = plt.imread(f'{PATH}valid/maluch/{files[2]}')
plt.imshow(img);

Oto jak wyglądają dane surowe.

In [0]:
img.shape

In [0]:
img[:4,:4]

## Nasz pierwszy model: szybki start


Użyjemy **wcześniej wytrenowanego** (ang. pre-trained) modelu, czyli modelu stworzonego przez kogoś innego dla rozwiązania innego problemu. Zamiast budować model od zera, aby rozwiązać podobny problem, jako punkt wyjścia wykorzystamy model wytrenowany na ImageNet (1,2 miliona obrazów i 1000 klas). Model ten to Convolutional Neural Network (CNN), rodzaj sieci neuronowej, którą używa się budując najnowocześniejsze modele rozpoznawania obrazów. Podczas tego kursu będziemy się uczyć wszystkiego o CNN.

Będziemy używać model **resnet34**. resnet34 to wersja modelu, który wygrał konkurs ImageNet 2015. Tutaj jest więcej informacji na temat [modeli resnet](https://github.com/KaimingHe/deep-residual-networks). Przyjrzymy się im dogłębnie później, ale na razie skupimy się na ich efektywnym wykorzystaniu.

Oto jak wytrenować i zewaluować model *old polish cars* przy pomocy 3 linii kodu i w mniej niż 20 sekund:

In [0]:
# Odkomentuj poniższe polecenie, jeśli chcesz zresetować cache-owane aktywacje
# shutil.rmtree(f'{PATH}tmp', ignore_errors=True)

In [0]:
arch=resnet34
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 3)

Jak dobry jest ten model? Kilka lat temu, przy podobnym zadaniu, osiągano maksymalnie 80% dokładności. Ale konkurs rozpoznawania obrazów ogłoszony przez Kaggle, spowodował ogromny skok do 98,9% dokładności, a autor pewnej popularnej biblioteki deep learning zwyciężył w tym konkursie.

## Analizowanie wyników: oglądanie zdjęć

Oprócz ogólnego spojrzenia na wskaźniki warto przyjrzeć się poniższym przykładom:

1. Kilka losowo wybranych poprawnych rozpoznań
2. Kilka losowo wybranych nieprawidłowych rozpoznań
3. Najbardziej poprawnie rozpoznania każdej klasy (tj. te z najwyższym prawdopodobieństwem z poprawnych)
4. Najbardziej niepoprawne rozpoznania każdej klasy (tj. te z najwyższym prawdopodobieństwem z nieprawidłowych)
5. Najbardziej niepewne rozpoznania (tj. te z prawdopodobieństwem najbliższym 0,5).

In [0]:
# To są etykiety dla danych walidacyjnych
data.val_y

In [0]:
# stąd wiemy, że "maluch" to etykieta dla 0, a "polonez" to etykieta dla 1
data.classes

In [0]:
# to tworzy prognozę dla zestawu walidacyjnego. Prognozy są w skali logarytmicznej
log_preds = learn.predict()
log_preds.shape

In [0]:
log_preds[:10]

In [0]:
preds = np.argmax(log_preds, axis=1)  # od logarytmu prawdopodobieństwa do 0 lub 1
probs = np.exp(log_preds[:,1])        # pr(polonez)

In [0]:
def rand_by_mask(mask): return np.random.choice(np.where(mask)[0], 4, replace=False)
def rand_by_correct(is_correct): return rand_by_mask((preds == data.val_y)==is_correct)

In [0]:
def plot_val_with_title(idxs, title):
    imgs = np.stack([data.val_ds[x][0] for x in idxs])
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(data.val_ds.denorm(imgs), rows=1, titles=title_probs)

In [0]:
def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

In [0]:
def load_img_id(ds, idx): return np.array(PIL.Image.open(PATH+ds.fnames[idx]))

def plot_val_with_title(idxs, title):
    imgs = [load_img_id(data.val_ds,x) for x in idxs]
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8))

In [0]:
# 1. Kilka losowo wybranych poprawnych rozpoznań
plot_val_with_title(rand_by_correct(True), "Correctly classified")

In [0]:
# 2. Kilka losowo wybranych nieprawidłowych rozpoznań
plot_val_with_title(rand_by_correct(False), "Incorrectly classified")

In [0]:
def most_by_mask(mask, mult):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:4]]

def most_by_correct(y, is_correct): 
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask(((preds == data.val_y)==is_correct) & (data.val_y == y), mult)

In [0]:
plot_val_with_title(most_by_correct(0, True), "Most correct maluchs")

In [0]:
plot_val_with_title(most_by_correct(1, True), "Most correct polonezes")

In [0]:
plot_val_with_title(most_by_correct(0, False), "Most incorrect maluchs")

In [0]:
plot_val_with_title(most_by_correct(1, False), "Most incorrect polonezes")

In [0]:
most_uncertain = np.argsort(np.abs(probs -0.5))[:4]
plot_val_with_title(most_uncertain, "Most uncertain predictions")

## Analizowanie wyników

### Tablica pomyłek

In [0]:
log_preds = learn.predict()
y = data.val_y
probs = np.exp(log_preds)
preds = np.argmax(probs, axis=1)
probs = probs[:,1]

Popularnym sposobem analizy wyniku modelu klasyfikacyjnego jest użycie [tablicy pomyłek (macierzy błędów)](http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/). Scikit-learn ma wygodną funkcję, którą możemy wykorzystać w tym celu:


In [0]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

Możemy po prostu wyświetlić macierz błędów tekstowo lub wykreślić graficznie (co jest przydatne głównie dla zależności z większą liczbą kategorii).

In [0]:
mpl.style.use('default')
mpl.style.use('seaborn-ticks')

plot_confusion_matrix(cm, data.classes)

## Zrozumienie kodu naszego pierwszego modelu

Spójrzmy na kod linia po linii.

**tfms** oznacza *transformacje*. `tfms_from_model` zajmuje się zmianą rozmiaru, kadrowaniem obrazu, wstępną normalizacją (tworzeniem danych z (mean,stdev) równym (0,1)), i nie tylko.

In [0]:
tfms = tfms_from_model(resnet34, sz)

Potrzebujemy **ścieżkę (PATH)** wskazującą na zbiór danych (dataset). W tej ścieżce będziemy również przechowywać tymczasowe dane i końcowe wyniki. `ImageClassifierData.from_paths` odczytuje dane z podanej ścieżki i tworzy zestaw danych gotowy na trening.

In [0]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms)

`ConvLearner.pretrained` buduje *ucznia*, który zawiera wcześniej wytrenowany model. Ostatnia warstwa modelu musi zostać zastąpiona warstwą odpowiednich wymiarów. Ten model został wytrenowany dla 1000 klas, a ostateczna warstwa prognozuje wektor z 1000-cem prawdopodobieństw. Model dla old polish cars musi oddać wektor dwuwymiarowy. Poniższy schemat pokazuje na przykładzie, w jaki sposób zostało to zrobione w jednej z najwcześniej odnoszących sukces sieci konwolucyjnych CNN. Warstwa "FC8" zostanie tutaj zastąpiona nową warstwą z 2 wyjściami.

<img src="https://github.com/fastai/fastai/raw/12558d73b0e9487251315204d1033e35df9f5053/courses/dl1/images/pretrained.png" width="500">
[original image](https://image.slidesharecdn.com/practicaldeeplearning-160329181459/95/practical-deep-learning-16-638.jpg)

In [0]:
learn = ConvLearner.pretrained(resnet34, data, precompute=True)

Uczymy się *parametrów* poprzez dopasowanie modelu do danych. *Hiperparametr* to inny rodzaj parametru, którego nie można bezpośrednio nauczyć się ze zwykłego procesu treningowego. Parametry te wyrażają właściwości modelu "wyższego poziomu", takie jak jego złożoność, czy jak szybko ma się uczyć. Dwa przykłady hiperparametrów to *tempo uczenia się* i *liczba epok*.

Podczas iteracyjnego trenowania sieci neuronowej, *wsad (batch)* lub *mini-wsad (mini-batch)* jest podzbiorem treningowych próbek użytych w jednej iteracji metody stochastycznego gradientu prostego, ang. Stochastic Gradient Descent (SGD). *Epoka* to pojedyncze przejście przez cały zestaw treningowy, na nią składa się wiele iteracji SGD.

Możemy teraz *dopasować (fit)* model; to znaczy, użyć *metody gradientu prostego (gradient descent)*, aby znaleźć najlepsze parametry dla w pełni połączonej warstwy (fully connected layer - FC), którą dodaliśmy, aby oddzielać zdjęcia maluchów od polonezów. Musimy podać dwa hipertermery: *tempo uczenia się (learning rate)* (zazwyczaj 1e-2 lub 1e-3 jest dobrym punktem wyjścia, później przyjrzymy się temu bliżej) i *liczbę epok* (możesz podać większą liczbę i po prostu przerwij trening, gdy zauważysz, że dokładność (accuracy) już się nie poprawia, a następnie uruchom ponownie trening z liczbą epok, którą ustaliłeś).

In [0]:
learn.fit(1e-2, 1)